In [1]:
import numpy as np
import pandas as pd

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d uciml/sms-spam-collection-dataset

Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
License(s): unknown
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 58.5MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/sms-spam-collection-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
!touch spam.csv

In [ ]:
import pandas as pd

# Upload the file
from google.colab import files
uploaded = files.upload()

# Check the filename in the uploaded dictionary
file = list(uploaded.keys())[0]

encodings = ['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']
for encoding in encodings:
    try:
        df = pd.read_csv(file, encoding=encoding)
        print(f"File successfully read with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")
        continue

if 'df' in locals():
    print("CSV file successfully loaded.")
else:
    print("Encoding failed.")


In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace =True )

In [ ]:
df.sample(5)

In [ ]:
df.rename(columns ={'v1' : 'TYPE','v2' : 'Mail'},inplace = True)

In [ ]:
df.sample(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df['TYPE'] =encoder.fit_transform(df['TYPE'])

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates(keep ='first')

In [ ]:
df.duplicated().sum()

In [ ]:
df.head()

In [ ]:
df['TYPE'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(df['TYPE'].value_counts(), labels = ['ham','spam'], autopct ="%.2f")
plt.show()

In [ ]:
import nltk

In [ ]:
!pip install nltk

In [ ]:
nltk.download('punkt')

In [ ]:
df['num_characters'] = df['Mail'].apply(len)

In [ ]:
df.head()

In [ ]:
df['num_words'] = df['Mail'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df['num_sentences'] = df['Mail'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df[['num_characters','num_words','num_sentences']].describe()

In [ ]:
df[df['TYPE']==0][['num_characters','num_words','num_sentences']].describe()

In [ ]:
df[df['TYPE']==1][['num_characters','num_words','num_sentences']].describe()

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.histplot(df[df['TYPE']==0]['num_characters'])
sns.histplot(df[df['TYPE']==1]['num_characters'], color='red')

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df[df['TYPE']==0]['num_words'])
sns.histplot(df[df['TYPE']==1]['num_words'], color='red')

In [ ]:
sns.pairplot(df, hue='TYPE')

In [ ]:
sns.heatmap(df.corr(),annot =True)

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
from nltk.corpus import stopwords

# Get the English stopwords
english_stopwords = set(stopwords.words('english'))


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

ps = PorterStemmer()

def transform_text(Mail):
    Mail = Mail.lower()
    Mail = nltk.word_tokenize(Mail)
    y = []
    for i in Mail:
        if i.isalnum():
             y.append(i)

    text = y[:]
    y.clear()

    for i in Mail:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    Mail = y[:]
    y.clear()

    for i in Mail:
        y.append(ps.stem(i))

    return " ".join(y)

transformed_text = transform_text("I'm gonna be home soon and i dont want to talk about this stuff anymore tonight,K?I've cried enough today.")
print(transformed_text)

In [ ]:
!pip show nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem('walking')

In [ ]:
df['transformed_text'] = df['Mail'].apply(transform_text)

In [ ]:
df.head()

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width = 550,height = 525,min_font_size = 10,background_color='red')


In [ ]:
spam_wc = wc.generate(df[df['TYPE']==1]['transformed_text'].str.cat(sep =" "))

In [ ]:
plt.figure(figsize=(15,6))
plt.imshow(spam_wc)

In [ ]:
ham_wc = wc.generate(df[df['TYPE']==0]['transformed_text'].str.cat(sep =" "))

In [ ]:
plt.figure(figsize=(15,6))
plt.imshow(ham_wc)

In [ ]:
df.head()

In [ ]:
spam_corpus =[]
for msg in df[df['TYPE']==1]['transformed_text'].tolist():
    spam_corpus.append(msg)

In [ ]:
len(spam_corpus)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv =CountVectorizer()
tfidf = TfidfVectorizer(max_features = 300)

In [ ]:
X = tfidf.fit_transform(df['transformed_text']).toarray()

In [ ]:
X.shape

In [ ]:
y =df['TYPE'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .2,random_state =2)

In [ ]:
!pip install xgboost

In [ ]:
!pip show scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc =SVC(kernel ='sigmoid',gamma = 1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
bc = BaggingClassifier(n_estimators= 50,random_state = 2)
etc = ExtraTreesClassifier(n_estimators= 50,random_state = 2)
gbdt = GradientBoostingClassifier(n_estimators= 50,random_state = 2)


In [ ]:
clfs = {
    'SVC': svc,
    'KN' : knc,
    'NB': mnb,
    'DT': dtc,
    'BgC': bc,
    'ETC': etc,
    'GBDT': gbdt
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  accuracy = accuracy_score(y_test,y_pred)
  precision = precision_score(y_test,y_pred)

  return precision,accuracy

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
train_classifier(svc,X_train,y_train,X_test,y_test)

In [ ]:
!pip install scikit-learn
import sklearn.metrics

In [ ]:
def train_classifier(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)

    if clf.score(X_test, y_test) > 0.5:

        accuracy = clf.score(X_test, y_test)
        precision = precision_score(y_test, clf.predict(X_test))


        return accuracy, precision
    else:

        return None



In [ ]:
accuracy_scores = []
precision_scores = []

for name, clf in clfs.items():
    current_accuracy, currrent_precision = train_classifier(clf, X_train, y_train, X_test, y_test)
    print("for ", name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)

    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

In [ ]:
performance_def = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores,'Precision': precision_scores}).sort_values('Precision',ascending = False)

In [ ]:
performance_def